In [55]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4176k  100 4176k    0     0  3558k      0  0:00:01  0:00:01 --:--:-- 19.4M


In [56]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::928808346782:role/SageMakerMLOpsRole
sagemaker-us-west-2-928808346782


In [4]:
cd ptuning

/home/ec2-user/SageMaker/ChatGLM/ptuning


In [5]:
!pwd

/home/ec2-user/SageMaker/ChatGLM/ptuning


In [6]:
# 下载 ADGEN 数据集
!wget -O AdvertiseGen.tar.gz https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1

--2023-05-24 02:52:43--  https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1
Resolving cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)... 166.111.6.101, 2402:f000:1:406:166:111:6:101
Connecting to cloud.tsinghua.edu.cn (cloud.tsinghua.edu.cn)|166.111.6.101|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cloud.tsinghua.edu.cn/seafhttp/files/1175ce70-0dbb-4622-9cf2-8e6e54977bee/AdvertiseGen.tar.gz [following]
--2023-05-24 02:52:44--  https://cloud.tsinghua.edu.cn/seafhttp/files/1175ce70-0dbb-4622-9cf2-8e6e54977bee/AdvertiseGen.tar.gz
Reusing existing connection to cloud.tsinghua.edu.cn:443.
HTTP request sent, awaiting response... 200 OK
Length: 17069994 (16M) [application/octet-stream]
Saving to: ‘AdvertiseGen.tar.gz’

100%[======================================>] 17,069,994  5.79MB/s   in 2.8s   

2023-05-24 02:52:47 (5.79 MB/s) - ‘AdvertiseGen.tar.gz’ saved [17069994/17069994]



In [7]:
!tar -xzvf AdvertiseGen.tar.gz

AdvertiseGen/
AdvertiseGen/train.json
AdvertiseGen/dev.json


In [8]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [9]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "THUDM/chatglm-6b"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    #allow_patterns=allow_patterns,
)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

In [19]:
print(model_download_path)
model_uri="s3://"+bucket+"/llm/models/chatglm/original/"
print(model_uri)

model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d
s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/


In [21]:
! ./s5cmd sync ./model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/ s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/

cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/README.md s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/README.md
cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/quantization.py s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/quantization.py
cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/config.json s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/config.json
cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/modeling_chatglm.py s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/modeling_chatglm.py
cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d/tokenizer_config.json s3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/tokenizer_config.json
cp model/models--THUDM--chatglm-6b/snapshots/a10da4c68b5d616030d3531fc37a13bb44ea814d

In [22]:
bucket

'sagemaker-us-west-2-928808346782'

In [23]:
!./s5cmd sync ./AdvertiseGen/ s3://{bucket}/llm/chatglm/datasets/

cp AdvertiseGen/dev.json s3://sagemaker-us-west-2-928808346782/llm/chatglm/datasets/dev.json
cp AdvertiseGen/train.json s3://sagemaker-us-west-2-928808346782/llm/chatglm/datasets/train.json


In [24]:
cd ../

/home/ec2-user/SageMaker/ChatGLM


In [40]:
!ls

chatglm-sagemaker-endpoint-byos.ipynb	      code
chatglm-sagemaker-finetune-p-tuning-v2.ipynb  ptuning


In [57]:
# define Training Job Name 
import time
from sagemaker.huggingface import HuggingFace

instance_type="ml.g4dn.2xlarge"

job_name = f'huggingface-chatglm-simple-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/ptune-adgen/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            : '50'
}

inputs = {
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}

In [42]:
environment

{'MODEL_S3_PATH': 's3://sagemaker-us-west-2-928808346782/llm/model/chatglm/original/',
 'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
 'TRAIN_DATASET': '/opt/ml/input/data/AdvertiseGen/train.json',
 'TEST_DATASET': '/opt/ml/input/data/AdvertiseGen/dev.json',
 'PROMPT_COLUMN': 'content',
 'RESPONSE_COLUMN': 'summary',
 'MODEL_NAME_OR_PATH': 'THUDM/chatglm-6b',
 'OUTPUT_DIR': '/opt/ml/model/adgen-chatglm-6b-ft',
 'MODEL_OUTPUT_S3_PATH': 's3://sagemaker-us-west-2-928808346782/llm/models/chatglm/ptune-adgen/',
 'TRAIN_STEPS': '50'}

In [43]:
inputs

{'AdvertiseGen': 's3://sagemaker-us-west-2-928808346782/llm/chatglm/datasets/'}

In [58]:
# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'sm_ptune_train.py',
    source_dir           = './ptuning',
    instance_type        = instance_type,
    instance_count       = instance_count,
    base_job_name        = job_name,
    role                 = role,
    script_mode          = True,
    transformers_version = '4.26',
    pytorch_version      = '1.13',
    py_version           = 'py39',
    environment = environment
)

In [50]:
role

'arn:aws:iam::928808346782:role/SageMakerMLOpsRole'

In [59]:
huggingface_estimator.fit(inputs=inputs)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-simple-2023-05-24-1-2023-05-24-12-31-12-810


2023-05-24 12:31:15 Starting - Starting the training job...
2023-05-24 12:31:31 Starting - Preparing the instances for training......
2023-05-24 12:32:29 Downloading - Downloading input data...
2023-05-24 12:32:49 Training - Downloading the training image.....................
2023-05-24 12:36:30 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-24 12:36:57,697 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-24 12:36:57,717 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-24 12:36:57,728 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-24 12:36:57,731 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-24 12:36:58,147 sagemaker-training-toolkit INFO     Installing dependenc